### Worksheet 08

Name:  Renming Zhang\
UID: U97677301

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [14]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[3.9752514874879807, 5.732337418344297, 5.194276242157175, 2.840540031772397, 6.305113966397156, 3.8599098562252303, 3.930438953253478, 4.960824374640213, 5.485999408828835, 5.708080245681613]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [15]:
c2 = np.random.normal( 8, 1 ,10).tolist()
print(c2)

[8.921160822253787, 8.748049889480912, 6.624357158908375, 8.944327929632252, 7.408652239035374, 8.636599635740051, 9.095013839084817, 8.08552338178954, 7.733841241058138, 7.123085404627725]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [16]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.708080245681613, 5.485999408828835, 4.960824374640213, 3.930438953253478, 3.8599098562252303, 6.305113966397156, 2.840540031772397, 5.194276242157175, 7.123085404627725, 5.732337418344297]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

#### 1.Number of Components (K): This is the number of Gaussian distributions in the mixture model. You mentioned there are 2 mixture components.

#### 2.Component Weights (π_k): These represent the probabilities of selecting each component. You would need to know the weights of the two components. For instance, π_1 and π_2, and they should sum to 1.

#### 3.Means (μ_k): Each component has its own mean, which represents the center of the Gaussian distribution for that component. You would need to know the mean values for both components, denoted as μ_1 and μ_2.

#### 4.Covariance Matrices (Σ_k): Each component has its own covariance matrix, which describes the shape and spread of the Gaussian distribution for that component. You would need to know the covariance matrices for both components, denoted as Σ_1 and Σ_2.

#### 5.Dimensionality (D): This represents the number of features or dimensions in your data. You would need to know the value of D.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [17]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[3.930438953253478, 3.8599098562252303, 2.840540031772397]
[5.708080245681613, 5.485999408828835, 4.960824374640213, 6.305113966397156, 5.194276242157175, 7.123085404627725, 5.732337418344297]
P(C_1) = 0.3,  P(C_2) = 0.7
mean_1 = 3.5436296137503684,  mean_2 = 5.7871024372395725
var_1 = 0.24799653906424932,  var_2 = 0.45532450222451354


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [18]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[1] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  5.708080245681613
probability of observing that point if it came from cluster 0 =  4.630104300966374e-17
probability of observing that point if it came from cluster 1 =  0.863075030803411
point =  5.485999408828835
probability of observing that point if it came from cluster 0 =  7.687209956067425e-14
probability of observing that point if it came from cluster 1 =  0.7040907644831222
point =  4.960824374640213
probability of observing that point if it came from cluster 0 =  1.303842017630707e-07
probability of observing that point if it came from cluster 1 =  0.16884664545652642
point =  3.930438953253478
probability of observing that point if it came from cluster 0 =  0.476642970290783
probability of observing that point if it came from cluster 1 =  0.00021478056146357583
point =  3.8599098562252303
probability of observing that point if it came from cluster 0 =  0.7133052479932452
probability of observing that point if it came from cluster 1 =  0.00011284270347016186
point = 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [19]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x), sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6996688913947523,  P(C_2) = 0.7001419036879633
mean_1 = 3.543455375156122,  mean_2 = 5.7867223568402695
var_1 = 0.24804936440477646,  var_2 = 0.45594498455967664


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [20]:
prob_c0_x = []
prob_c1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

5.708080245681613
Probability of coming from C_1 = 5.4206283753106104e-17
Probability of coming from C_2 = 1.0

5.485999408828835
Probability of coming from C_1 = 1.0993409147111151e-13
Probability of coming from C_2 = 0.9999999999998901

4.960824374640213
Probability of coming from C_1 = 7.702180188446873e-07
Probability of coming from C_2 = 0.9999992297819812

3.930438953253478
Probability of coming from C_1 = 0.9995376819087685
Probability of coming from C_2 = 0.0004623180912314768

3.8599098562252303
Probability of coming from C_1 = 0.9998373434913729
Probability of coming from C_2 = 0.00016265650862714055

6.305113966397156
Probability of coming from C_1 = 4.249283603042448e-27
Probability of coming from C_2 = 1.0

2.840540031772397
Probability of coming from C_1 = 0.9999999741190138
Probability of coming from C_2 = 2.588098623624625e-08

5.194276242157175
Probability of coming from C_1 = 1.0300553535618809e-09
Probability of coming from C_2 = 0.9999999989699445

7.123085404627725

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [21]:
labels = [0 if prob_c0_x[i] > prob_c1_x[i] else 1 for i in range(len(data))]
print(labels)

[1, 1, 1, 0, 0, 1, 0, 1, 1, 1]
